In [40]:
# %load_ext dotenv
%dotenv

In [8]:
import os
from neo4j import GraphDatabase, Result

driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI"),
    auth=(os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))
)

driver.verify_connectivity()

In [29]:
movies = driver.execute_query("""
MATCH (m:Movie) WHERE m.plot IS NOT NULL
RETURN m.movieId AS movieId, m.title AS title, m.plot AS plot
LIMIT 1000
""",
result_transformer_=Result.to_df)

len(movies)


9083

In [11]:
movies[0:10]

,movieId,title,plot
0,1,Toy Story,A cowboy doll is profoundly threatened and jea...
1,2,Jumanji,When two kids find and play a magical board ga...
2,3,Grumpier Old Men,John and Max resolve to save their beloved bai...
3,4,Waiting to Exhale,"Based on Terry McMillan's novel, this film fol..."
4,5,Father of the Bride Part II,"In this sequel, George Banks deals not only wi..."
5,6,Heat,A group of professional bank robbers start to ...
6,7,Sabrina,An ugly duckling having undergone a remarkable...
7,8,Tom and Huck,Tom and Huck witness Injun Joe's killing of Do...
8,9,Sudden Death,A former fireman takes on a group of terrorist...
9,10,GoldenEye,James Bond teams up with the lone survivor of ...


In [30]:
from requests import post
from json import dumps, loads

embeddings = []

for _, n in movies.iterrows():
    res = post("http://localhost:11434/api/embeddings", dumps({
        "model": "orca-mini",
        "prompt": f"{n['title']}: {n['plot']}"
    }))

    json = loads(res.content)

    embeddings.append({"movieId": n['movieId'], "embedding": json['embedding']})

In [31]:
import pandas as pd

embedding_df = pd.DataFrame(embeddings)

embedding_df.head()


,movieId,embedding
0,1,"[0.41867926716804504, -3.812939167022705, -0.1..."
1,2,"[-0.438289076089859, -1.320231556892395, 0.694..."
2,3,"[-1.5189074277877808, -0.8654581904411316, -0...."
3,4,"[-1.209105372428894, -0.10715539753437042, 0.2..."
4,5,"[-0.9011550545692444, -1.474434733390808, 0.19..."


In [33]:
embedding_df.to_csv('data/embeddings.csv', index=False)

In [25]:
# merge movies and embeddings_df on movieId
combined_df = movies.merge(embedding_df, on="movieId")

combined_df.head()


,movieId,title,plot,embedding
0,1,Toy Story,A cowboy doll is profoundly threatened and jea...,"[0.41867926716804504, -3.812939167022705, -0.1..."
1,2,Jumanji,When two kids find and play a magical board ga...,"[-0.438289076089859, -1.320231556892395, 0.694..."
2,3,Grumpier Old Men,John and Max resolve to save their beloved bai...,"[-1.5189074277877808, -0.8654581904411316, -0...."
3,4,Waiting to Exhale,"Based on Terry McMillan's novel, this film fol...","[-1.209105372428894, -0.10715539753437042, 0.2..."
4,5,Father of the Bride Part II,"In this sequel, George Banks deals not only wi...","[-0.9011550545692444, -1.474434733390808, 0.19..."


In [28]:
len(embeddings[0]['embedding'])


3200

In [38]:
prompt = f"Find me a film that depicts a young man's journey through many historical events in search of true love."

res = post("http://localhost:11434/api/embeddings", dumps({
    "model": "orca-mini",
    "prompt": prompt
}))

json = loads(res.content)



prompt_df = pd.DataFrame([{"prompt": prompt, "embedding": json['embedding']}])


prompt_df.to_csv('data/prompt.csv', index=False)


In [48]:
llm(template.format(fruit="apple"))

"\nApples and pears, guv'nor! Got some nice 'andsome ones in, they'd look lovely on yer mantelpiece!"

In [59]:
from langchain.chat_models.openai import ChatOpenAI
chat_llm = ChatOpenAI()


from langchain.prompts import SystemMessagePromptTemplate

instructions = SystemMessagePromptTemplate.from_template("""
You are a {location} surfer dude, having a conversation about the surf conditions on the beach.
Respond using surfer slang.
""")


from langchain.schema import HumanMessage, SystemMessage

question  = HumanMessage(content="What is the weather like?")


AIMessage(content="Dude, the weather is totally gnarly! It's sunny, with a few clouds cruisin' by. The breeze is blowin' offshore, so the waves are lookin' pretty sick. It's gonna be a rad day out there!", additional_kwargs={}, example=False)